In [55]:
import neurokit2
import pandas as pd
import matplotlib.pyplot as plt

def get_key_timestamps(target, duration_ms, df, df_key='timestamp'):
    target_before = target - duration_ms
    target_after = target + duration_ms

    timestamp_before = 0
    timestamp_target = 0
    timestamp_after = 0

    # premise: df[df_key] is sorted
    for df_index, df_row in df.iterrows():
        if df_row[df_key] <= target_before:
            timestamp_before = df_row[df_key]
        elif df_row[df_key] <= target:
            timestamp_target = df_row[df_key]
        elif df_row[df_key] >= target_after:
            timestamp_after = df_row[df_key]
            break
    return timestamp_before, timestamp_target, timestamp_after

In [56]:
plt.style.use('ggplot')

time_middle_file_path = 'middle-files/'
hrv_output_path = 'hrv-middle-data-test/'

p_index = 0


participant_key = 'P' + str(p_index + 1).zfill(2)

df_time = pd.read_csv(f'{time_middle_file_path}{participant_key}_app_usage_events_with_confidences.csv')

In [57]:
df_hrv = pd.read_csv(f'dataset/{participant_key}/RRI.csv')

df_analyse = pd.DataFrame()
df_analyse['RRI'] = df_hrv.interval
df_analyse['RRI_Time_ms'] = df_hrv.timestamp
df_analyse['RRI_Time'] = df_hrv.timestamp / 1000

# premise of the hrv extracting data function
df_analyse.sort_values(by='RRI_Time_ms')



,RRI,RRI_Time_ms,RRI_Time
0,580.720,1557278021425,1.557278e+09
1,580.720,1557278022004,1.557278e+09
2,497.760,1557278022647,1.557278e+09
3,431.392,1557278023558,1.557278e+09
4,663.680,1557278024197,1.557278e+09
...,...,...,...
250085,779.824,1557838533555,1.557839e+09
250086,846.192,1557838534168,1.557839e+09
250087,829.600,1557838534775,1.557839e+09
250088,879.376,1557838535968,1.557839e+09


In [58]:
test_minutes = 2

# filter out use time that is less than 1 minute
loc_time = df_time.loc[(df_time.time_difference > test_minutes * 60 * 1000) & (df_time['confidenceStill'] > 0.4)]
loc_time

,name,packageName,foreground_time,background_time,time_difference,UNLOCK_timestamp,OFF_timestamp,confidenceStill,confidenceUnknown,confidenceOnFoot,confidenceWalking,confidenceInVehicle,confidenceOnBicycle,confidenceRunning,confidenceTilting
13,카카오톡,com.kakao.talk,1557317900079,1557318132533,232454,1557317900011,1557319911763,0.4337,0.0587,0.0843,0.0843,0.1546,0.0645,0.0111,0.1010
22,카카오톡,com.kakao.talk,1557408596729,1557408719867,123138,1557408595111,1557408743143,0.1263,0.2366,0.1497,0.1497,0.0909,0.0676,0.0606,0.2053
25,카카오톡,com.kakao.talk,1557409127927,1557409270627,142700,1557408901577,1557409367260,0.1608,0.0846,0.0682,0.0682,0.5716,0.0337,0.0198,0.0000
92,카카오톡,com.kakao.talk,1557649532133,1557649899940,367807,1557649504769,1557650180973,0.0327,0.0602,0.7206,0.7206,0.0214,0.0163,0.0135,0.1207
93,카카오톡,com.kakao.talk,1557649982791,1557650178403,195612,1557649504769,1557650180973,0.1002,0.2041,0.4566,0.4566,0.0790,0.0474,0.0424,0.0000
111,카카오톡,com.kakao.talk,1557764018715,1557764166585,147870,1557763029096,1557764280356,0.4206,0.1048,0.1060,0.1060,0.1556,0.0710,0.0243,0.0525
113,카카오톡,com.kakao.talk,1557764774047,1557764919571,145524,1557764655115,1557765331366,0.4473,0.0754,0.0613,0.0613,0.1589,0.0382,0.0161,0.1421
132,카카오톡,com.kakao.talk,1557841648801,1557841771978,123177,1557841463415,1557841967644,0.1025,0.1575,0.0438,0.0438,0.4400,0.0400,0.0375,0.1250


In [60]:
res = None
for index, row in loc_time.iterrows():
    foreground_time = row['foreground_time']
    background_time = row['background_time']

    (ts_before, ts_target, ts_after) = get_key_timestamps(foreground_time, test_minutes * 60 * 1000, df_analyse,
                                                          df_key='RRI_Time_ms')

    if ts_before > 0 and ts_target > 0 and ts_after > 0:
        temp = df_analyse.loc[(df_analyse['RRI_Time_ms'] >= ts_before) & (df_analyse['RRI_Time_ms'] <= ts_target)]
        
        before_res = neurokit2.hrv_time(
            df_analyse.loc[(df_analyse['RRI_Time_ms'] >= ts_before) & (df_analyse['RRI_Time_ms'] <= ts_target)]
        )

        after_res = neurokit2.hrv_time(
            df_analyse.loc[(df_analyse['RRI_Time_ms'] > ts_target) & (df_analyse['RRI_Time_ms'] <= ts_after)]
        )

        res_row = pd.merge(before_res, after_res, suffixes=('_before', '_after'), left_index=True, right_index=True)
        res_row['foreground_time'] = foreground_time

        if res is None:
            res = res_row
        else:
            res = pd.concat([res, res_row], ignore_index=True)

res.to_csv(hrv_output_path + participant_key + '_hrv_result.csv', index=False)
print(res)

OSError: Cannot save file into a non-existent directory: 'hrv-middle-data-test'